### Import all the Relevant Data and Packages

In [1]:
import sib_main.sib_sub.sib_validation_main as svm
import sib_main.sib_sub.sib_validation_imports as svi

import copy
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, Point
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.multipoint import MultiPoint
from shapely.ops import nearest_points
from datetime import datetime

ground_truth = gpd.read_file("E:/SIB/validation/test_data/ground_truth.shp")
model_det = gpd.read_file("E:/SIB/validation/test_data/model_detected.shp")
VHRsample = gpd.read_file("E:/SIB/validation/test_data/VHR_parcel.shp")

export_path = "E:/SIB/validation/test_data/"

### Preprocess the Data

In [2]:
# Preprocess the data

# 1) Confidence Filter 
svi.confidence_filter(model_det)

# 2) Change 'geometry' column from 'MULTIPOINT' to 'POINT'
ground_truth = svi.multi_to_single(ground_truth)
model_det = svi.multi_to_single(model_det)

# 3) Preprocess the data
ground_truth, model_det = svm.preprocessing(ground_truth, model_det, conf_percentile=10)
print("Number of Ground Truth Points: {}".format(len(ground_truth)))
print("Number of Model Detected Points: {}".format(len(model_det)))

# 4) Find maximum number of points within a gt_buffer
n_ = svi.n_max(ground_truth, model_det)
print(n_)

90.0% of values above = 0.205318
Accepted Geometries: 'POINT'
Accepted Geometries: 'POINT'
90.0% of values above = 0.205318
Number of Ground Truth Points: 38
Number of Model Detected Points: 31
2


### Test the True Positives

In [3]:
# Find the True Positives in the parcel

# 1) Call Calculate Nearest
# nearest_neighbour = call_calculate_nearest(ground_truth_points, model_detected_points):
nearest_neighbour = svm.call_calculate_nearest(ground_truth, model_det)
    
# 2) Call Calculate Distance
# nearest_neighbour = calculate_distance(nearest_neighbour):
nearest_neighbour = svm.calculate_distance(nearest_neighbour)

# 3) Call Calculate TP
# nearest_neighbour, tp = calculate_tp(nearest_neighbour):
nearest_neighbour, tp = svm.calculate_tp(nearest_neighbour)

# Need to check now whether we get all six True Positives out of this.
# To do so we will need to drop the indices of found True Positives within model_det and ground_truth.

gt_copy = copy.copy(ground_truth)
md_copy = copy.copy(model_det)
tp_first_pass = copy.copy(tp)

print("Length of Model det Points originally: {}".format(len(md_copy)))
print("Length of Ground Truth Points originally: {}".format(len(gt_copy)))
print("Length of TP Points originally: {}".format(len(tp_first_pass)))
print(type(tp))

C:\Users\alexk\anaconda3\envs\sib_validation\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Initial True Positives, Count: 6
Length of Model det Points originally: 31
Length of Ground Truth Points originally: 38
Length of TP Points originally: 6
<class 'geopandas.geodataframe.GeoDataFrame'>


### Test the False Positives

In [4]:
# Find the False Positives

# Find the False Positives in the parcel

# 1) Call Calculate Nearest
# nearest_neighbour = call_calculate_nearest(ground_truth_points, model_detected_points):
nearest_neighbour = svm.call_calculate_nearest(ground_truth, model_det)
    
# 2) Call Calculate Distance
# nearest_neighbour = calculate_distance(nearest_neighbour):
nearest_neighbour = svm.calculate_distance(nearest_neighbour)

# 3) Call Calculate FP
# nearest_neighbour, fp = calculate_fp(nearest_neighbour):
nearest_neighbour, fp = svm.calculate_fp(nearest_neighbour)

C:\Users\alexk\anaconda3\envs\sib_validation\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Initial False Positives, Count: 24


In [5]:
print("length of FP1: {}".format(len(fp)))
print("Type of FP1: {}".format(type(fp)))

length of FP1: 24
Type of FP1: <class 'geopandas.geodataframe.GeoDataFrame'>


In [6]:
# False positives outside 2m buffer
fp2 = svi.extract_points_outside_2m(ground_truth, model_det)
print("Type of FP2 is: {}".format(type(fp2)))
print("Length of FP2 is: {}".format(len(fp2)))

Type of FP2 is: <class 'geopandas.geodataframe.GeoDataFrame'>
Length of FP2 is: 1


### Test the Drop indices

In [ ]:
gt_copy = copy.copy(ground_truth)
md_copy = copy.copy(model_det)
tp_first_pass = copy.copy(tp)
fp_first_pass = copy.copy(fp)

In [ ]:
# Now we must drop the indiced and see whats left. 

print("Length of Model det Points originally: {}".format(len(md_copy)))
print("Length of Ground Truth Points originally: {}".format(len(gt_copy)))
print("Length of TP Points originally: {}".format(len(tp_first_pass)))
print("Length of FP Points originally: {}".format(len(fp_first_pass)))

md_copy = svi.drop_indices(md_copy, tp_first_pass, outcome_value='true')
md_copy = svi.drop_indices(md_copy, fp_first_pass, outcome_value='false')
gt_copy = svi.drop_indices(gt_copy, tp_first_pass, outcome_value='ground')


In [ ]:
print("Length of Model det Points New: {}".format(len(md_copy)))
print("Length of Ground Truth Points New: {}".format(len(gt_copy)))
print("Length of TP Points originally: {}".format(len(tp_first_pass)))
print("Length of FP Points originally: {}".format(len(fp_first_pass)))

In [ ]:
print("Length of Model det Points originally: {}".format(len(md_copy)))
print("Length of Ground Truth Points originally: {}".format(len(gt_copy)))
print("Length of TP Points originally: {}".format(len(tp_first_pass)))
print("Length of FP Points originally: {}".format(len(fp_first_pass)))

for i in range(0, 6):
    
    md_copy = svi.drop_indices(md_copy, tp_first_pass, outcome_value='true')
    md_copy = svi.drop_indices(md_copy, fp_first_pass, outcome_value='false')
    gt_copy = svi.drop_indices(gt_copy, tp_first_pass, outcome_value='ground')
    

In [ ]:
print("Length of Model det Points New: {}".format(len(md_copy)))
print("Length of Ground Truth Points New: {}".format(len(gt_copy)))
print("Length of TP Points originally: {}".format(len(tp_first_pass)))
print("Length of FP Points originally: {}".format(len(fp_first_pass)))

### Test the create_global_GeoDataFrame

In [7]:
# Test the global GeoDataFrame

global_fp_list = []

fp1_dict = fp.to_dict('list')
global_fp_list.append(fp1_dict)

fp2_dict = fp2.to_dict('list')
global_fp_list.append(fp2_dict)


In [8]:
len(global_fp_list)

2

In [9]:
global_fp_df = svi.create_global_GeoDataFrame(global_fp_list)

C:\Users\alexk\anaconda3\envs\sib_validation\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [10]:
print("Total False Positives inside buffer: {}".format(len(fp)))
print("Total False Positives outside buffer: {}".format(len(fp2)))
print("Total amount of False Positives in Global GeoDataFrame: {}".format(len(global_fp_df)))

Total False Positives inside buffer: 24
Total False Positives outside buffer: 1
Total amount of False Positives in Global GeoDataFrame: 25


### Test the False Negatives

In [11]:
# False Negatives

gt_copy = copy.copy(ground_truth)
md_copy = copy.copy(model_det)
print("Length of TP: {}".format(len(tp)))
print("Length of FP: {}".format(len(global_fp_df)))
print("\n")

Length of TP: 6
Length of FP: 25




In [12]:
print("Length of Ground Truth originally: {}".format(len(ground_truth)))

gt_copy = svi.drop_indices(gt_copy, tp, outcome_value='ground')
print("Length of Ground Truth after TP dropped: {}".format(len(gt_copy)))

gt_copy = svi.drop_indices(gt_copy, global_fp_df, outcome_value='ground')
print("Length of Ground Truth after FP dropped: {}".format(len(gt_copy)))


md_copy = svi.drop_indices(md_copy, tp, outcome_value='true')
md_copy = svi.drop_indices(md_copy, global_fp_df, outcome_value='false')


Length of Ground Truth originally: 38
Length of 'Ground Truth points' after TP rows dropped: 32

Length of Ground Truth after TP dropped: 32
Length of 'Ground Truth points' after FP rows dropped: 8

Length of Ground Truth after FP dropped: 8
Length of 'model_det' after TP rows dropped: 25

Length of 'model_det' after FP rows dropped: 0



In [13]:
fn = svm.calculate_fn(gt_copy, md_copy)

In [14]:
fn

,class,gt_ID,geometry,n
13,sheep,13,"POLYGON ((274379.003 396456.500, 274378.993 39...",0
24,sheep,24,"POLYGON ((274385.753 396440.250, 274385.743 39...",0
25,sheep,25,"POLYGON ((274384.253 396441.250, 274384.243 39...",0
26,sheep,26,"POLYGON ((274389.503 396439.000, 274389.493 39...",0
28,sheep,31,"POLYGON ((274454.753 396521.750, 274454.743 39...",0
29,sheep,32,"POLYGON ((274442.503 396515.500, 274442.493 39...",0
30,sheep,34,"POLYGON ((274449.503 396515.750, 274449.493 39...",0
31,sheep,36,"POLYGON ((274449.753 396514.250, 274449.743 39...",0


In [15]:
svm.export_GeoDataFrame(export_path, fn, file_name="fn", extension='.shp')

Exported to Local Drive
